In [1]:
import numpy as np
import pandas as pd
#import torch
from math import factorial
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [2]:
def pos(i,t,prod):
    '''
    Compute positive function and gradient information
    
    input:
        i - index of function
        t - iteration
        prod - wt*xt
        
    output:
        fpt - positive function value
        gfpt - positive function gradient
    '''
    fpt = 0.0 
    gfpt = 0.0 
    fpt = (L/2+prod)**i # no xt yet!
    gfpt = i*(L/2+prod)**(i-1) 
    return fpt,gfpt               

In [3]:
def comb(n, k):
    '''
    Compute combination
    
    input:
        n - total number
        k - number of chosen
    
    output:
        c - number of combination
    '''
    return factorial(n) / factorial(k) / factorial(n - k)

In [38]:
def neg(loss,i,t,prod):
    '''
    Compute negative function and gradient information
    
    input:
        loss - loss function
        i - index of function
        t - iteration
        prod - wt*xt
        
    output:
        fnt - negative function value
        gfnt - negative function gradient
    '''
    fnt = 0.0 # n stands for negative
    gfnt = 0.0
    for k in range(i,N+1):
        # compute forward difference
        delta = 0.0
        for j in range(k+1):
            delta += (-1)**(k-j)*comb(k,j)*loss(j/N)
        # compute coefficient
        beta = comb(N,k)*comb(k,i)*(N+1)*delta/(2*L)**k
        # compute function value
        fnt += beta*(L/2-prod)**(k-i)
        # compute gradient
        gfnt += beta*(k-i)*(L/2-prod)**(k-i-1)  # no xt yet!
    return fnt,gfnt

In [5]:
def p_hat(t,yt,ptm1):
    '''
    Approximate probability
    
    input:
        t - iteration
        yt - label at t
        ptm1 - p at t-1
    
    output:
        pt - p at t
    '''
    pt = (t*ptm1 + (yt+1)/2)/(t+1) # m stands for minus
    return pt

In [6]:
def a_hat(t,fpt,yt,ptm1,atm1):
    '''
    Approximate primal a
    
    input:
        t - iteration
        fpt - positive function at t
        yt - sample label at t
        ptm1 - p at t-1
        atm1 - a at t-1
    
    output:
        at - a at t
    '''
    at = (fpt*((yt+1)/2) + t*ptm1*atm1)/(t+1) # do not update pt yet!
    return at

In [7]:
def b_hat(t,fmt,yt,ptm1,btm1):
    '''
    Approximate primal b
    
    input:
        t - iteration
        fmt - negative function at t
        yt - sample label at t
        ptm1 - p at t-1
        btm1 - b at t-1
    
    output:
        bt - b at t-1
    '''
    bt = (fmt*((-yt+1)/2) + t*(1-ptm1)*btm1)/(t+1) # indicator of y=-1!
    return bt

In [8]:
def alpha_step(t,at,bt):
    '''
    Compute dual alpha
    
    input:
        t - iteration
        at - a at t
        bt - b at t
        
    output:
        alphat - alpha at t
    '''
    alphat = at + bt
    return alphat

In [9]:
def w_grad(fpt,gfpt,fnt,gfnt,yt,pt,at,bt,alphat):
    '''
    Gradient with respect to w
    
    input:
        fpt - positive function at t
        gfpt - positive function gradient at t
        fnt - negative function at t
        gfnt - negative function gradient at t
        yt - sample label at t
        pt - p at t
        at - a at t
        bt - b at t
        alphat - alpha at t
    output:
        gradwt - gradient w.r.t. w at t
    '''
    gradwt = 0.0
    if yt == 1:
        gradwt = (2*alphat*(1-pt) + 2*(1-pt)*(fpt-at) - 2*(1-pt)*fpt)*gfpt
    else:
        gradwt = (2*alphat*pt + 2*pt*(fnt-bt) - 2*pt*fnt)*gfnt
    return gradwt

In [42]:
def proj(wt,R):
    '''
    Projection
    
    input:
        wt - w at t
        R - radius
        
    output:
        proj - projected wt
    '''
    norm = np.linalg.norm(wt)
    if norm > R:
        wt = wt/norm*R
    return wt

In [11]:
def a_grad(fpt,yt,pt,at):
    '''
    Gradient with respect to a
    
    input:
        fpt - positive function at t
        yt - sample label at t
        pt - p at t
        at - a at t
    
    output:
        gradat - gradient w.r.t a at t
    '''
    gradat = 0.0 
    if yt == 1:
        gradat = -2*(1-pt)*(fpt-at)
    else:
        pass
    return gradat

In [12]:
def b_grad(fnt,yt,pt,bt):
    '''
    Gradient with respect to b
    
    input:
        fnt - negative function at t
        yt - sample label at t
        pt - p at t
        bt - b at t
    
    output:
        gradbt - gradient w.r.t b at t
    '''
    gradbt = 0.0 
    if yt == 1:
        pass
    else:
        gradbt = -2*pt*(fnt-bt)
    return gradbt

In [13]:
def obj(pt,fpt,fnt,at,bt,alphat,yt):
    '''
    Compute objective function value
    
    input:
        t - iteration
        pt - 
        wt - 
    
    output:
        F - objective funciton value
    '''
    F = 0.0
    if yt == 1:
        F = -pt*(1-pt)*alphat**2 + 2*alphat*(1-pt)*fpt+(1-pt)*(fpt-at)**2 - (1-pt)*fpt**2
    else:
        F = -pt*(1-pt)*alphat**2 + 2*alphat*pt*fnt+pt*(fnt-bt)**2 - pt*fnt**2
    return F

In [133]:
def SOLAM(T,epoch,loss, F = 0.0,wt = 0.0,pt = 0.0, at = 0.0,bt = 0.0,alphat = 0.0):
    '''
    Stochastic Online AUC Maximization
    
    input:
        T - total number of iteration
        F - objective function value
        loss - loss function
        pt - p at t
        wt - w at t
        at - a at t
        bt - b at t
        alphat - alpha at t
    output:
        W - record of each wt
        A - record of each at
        B - record of each bt
        ALPHA - record of each alphat
    '''
    
    # Initialize W
    P = np.zeros(T)
    W = np.zeros([T,d])
    A = np.zeros(T)
    B = np.zeros(T)
    ALPHA = np.zeros(T)
    
    # Loop
    for t in range(T*epoch,T*(epoch+1)):
        
        if t%1 == 0:
            print('iteration: %d objective: %f' % (t+1,F/(t+1)))
        # Update pt
        pt = p_hat(t,y[t-T*epoch],pt)
        # Update wt,at,bt
        prod = np.inner(wt,x[t-T*epoch])
        fpt = np.zeros(N+1)
        gfpt = np.zeros(N+1)
        fnt = np.zeros(N+1)
        gfnt = np.zeros(N+1)
        gradwt = 0.0
        gradat = 0.0
        gradbt = 0.0
        
        for i in range(N+1): # add up info of each i
            fpt[i],gfpt[i] = pos(i,t,prod) # partial info
            fnt[i],gfnt[i] = neg(loss,i,t,prod)
            gradwt += w_grad(fpt[i],gfpt[i],fnt[i],gfnt[i],y[t-T*epoch],pt,at,bt,alphat)
            gradat += a_grad(fpt[i],y[t-T*epoch],pt,at)
            gradbt += b_grad(fnt[i],y[t-T*epoch],pt,bt)
            F += obj(pt,fpt[i],fnt[i],at,bt,alphat,y[t-T*epoch])
        
        eta = 1/np.sqrt(t+1)/2
        wt -= eta*gradwt*y[t-T*epoch]*x[t-T*epoch]/(N+1) # step size as 1/t gradient descent
        wt = proj(wt,1)
        at -= eta*gradat/(N+1)
        bt -= eta*gradbt/(N+1)
        alphat = at+bt
        
        P[t-T*epoch] = pt
        W[t-T*epoch,:] = wt
        A[t-T*epoch] = at
        B[t-T*epoch] = bt
        ALPHA[t-T*epoch] = alphat    
        
    return F,P,W,A,B,ALPHA

In [134]:
def loader(filename):
    '''
    Data file loader
    
    input:
        filename - filename
    
    output:
        x - sample features
        y - sample labels
    '''
    # raw data
    raw_df = pd.read_csv(filename,header=None,sep = '\s+|:',engine='python')
    y = raw_df[0].values
    x = raw_df[raw_df.columns[2::2]].values
    # avoid nan
    x = np.nan_to_num(x)
    # normalize
    norm = np.linalg.norm(x,axis=1)
    x = x/norm[:,None]
    return x,y

In [135]:
x,y = loader('diabetes')
T,d = x.shape
L = 2 # range
N = 10 # degree

In [146]:
hinge = lambda x:max(0,1+L-2*L*x)
logistics = lambda x:np.log(1+np.exp(L-2*L*x))

F = 0.0
P = np.zeros(T)
W = np.zeros([T,d])
A = np.zeros(T)
B = np.zeros(T)
ALPHA = np.zeros(T)
w = np.zeros(d)

epochs = 100
roc_auc = np.zeros(epochs)
for epoch in range(epochs):
    F,P,W,A,B,ALPHA = SOLAM(T,epoch,hinge,F = F,wt = W[T-1,:],pt = P[T-1], at = A[T-1],bt = B[T-1],alphat = ALPHA[T-1])
    w = (T*epoch*w+np.sum(W,axis=0))/(T*(epoch+1))
    fpr, tpr, _ = roc_curve(y, np.dot(x,W[T-1]))
    roc_auc[epoch] = auc(fpr, tpr)

iteration: 1 objective: 0.000000
iteration: 2 objective: 0.000000
iteration: 3 objective: 0.000000
iteration: 4 objective: 0.581920
iteration: 5 objective: 0.783525
iteration: 6 objective: 1.125074
iteration: 7 objective: 1.068570
iteration: 8 objective: 1.176465
iteration: 9 objective: 1.096486
iteration: 10 objective: 1.237770
iteration: 11 objective: 1.272276
iteration: 12 objective: 1.159538
iteration: 13 objective: 1.166315
iteration: 14 objective: 1.117994
iteration: 15 objective: 1.197041
iteration: 16 objective: 1.175765
iteration: 17 objective: 1.152535
iteration: 18 objective: 1.128958
iteration: 19 objective: 1.084282
iteration: 20 objective: 1.029218
iteration: 21 objective: 0.995509
iteration: 22 objective: 0.944750
iteration: 23 objective: 0.892040
iteration: 24 objective: 0.878361
iteration: 25 objective: 0.862704
iteration: 26 objective: 0.856404
iteration: 27 objective: 0.837254
iteration: 28 objective: 0.813845
iteration: 29 objective: 0.779739
iteration: 30 objective

iteration: 490 objective: 0.187301
iteration: 491 objective: 0.186716
iteration: 492 objective: 0.185952
iteration: 493 objective: 0.185201
iteration: 494 objective: 0.184434
iteration: 495 objective: 0.185526
iteration: 496 objective: 0.184889
iteration: 497 objective: 0.184246
iteration: 498 objective: 0.183459
iteration: 499 objective: 0.182700
iteration: 500 objective: 0.184110
iteration: 501 objective: 0.183338
iteration: 502 objective: 0.182565
iteration: 503 objective: 0.181819
iteration: 504 objective: 0.183760
iteration: 505 objective: 0.182990
iteration: 506 objective: 0.182244
iteration: 507 objective: 0.181558
iteration: 508 objective: 0.182630
iteration: 509 objective: 0.181873
iteration: 510 objective: 0.181166
iteration: 511 objective: 0.180566
iteration: 512 objective: 0.182149
iteration: 513 objective: 0.181391
iteration: 514 objective: 0.180778
iteration: 515 objective: 0.180012
iteration: 516 objective: 0.179253
iteration: 517 objective: 0.180222
iteration: 518 objec

iteration: 966 objective: 0.177226
iteration: 967 objective: 0.177793
iteration: 968 objective: 0.178421
iteration: 969 objective: 0.179025
iteration: 970 objective: 0.178642
iteration: 971 objective: 0.178242
iteration: 972 objective: 0.177860
iteration: 973 objective: 0.177472
iteration: 974 objective: 0.177134
iteration: 975 objective: 0.176746
iteration: 976 objective: 0.177619
iteration: 977 objective: 0.178235
iteration: 978 objective: 0.177854
iteration: 979 objective: 0.178454
iteration: 980 objective: 0.178075
iteration: 981 objective: 0.177718
iteration: 982 objective: 0.177400
iteration: 983 objective: 0.177961
iteration: 984 objective: 0.178573
iteration: 985 objective: 0.179445
iteration: 986 objective: 0.180022
iteration: 987 objective: 0.179628
iteration: 988 objective: 0.180265
iteration: 989 objective: 0.180790
iteration: 990 objective: 0.181620
iteration: 991 objective: 0.182336
iteration: 992 objective: 0.182011
iteration: 993 objective: 0.181717
iteration: 994 objec

iteration: 1201 objective: 0.187799
iteration: 1202 objective: 0.187490
iteration: 1203 objective: 0.187164
iteration: 1204 objective: 0.186872
iteration: 1205 objective: 0.187363
iteration: 1206 objective: 0.187101
iteration: 1207 objective: 0.186778
iteration: 1208 objective: 0.186467
iteration: 1209 objective: 0.186158
iteration: 1210 objective: 0.186601
iteration: 1211 objective: 0.186296
iteration: 1212 objective: 0.185983
iteration: 1213 objective: 0.186442
iteration: 1214 objective: 0.186814
iteration: 1215 objective: 0.187720
iteration: 1216 objective: 0.187406
iteration: 1217 objective: 0.187114
iteration: 1218 objective: 0.187559
iteration: 1219 objective: 0.187241
iteration: 1220 objective: 0.186939
iteration: 1221 objective: 0.187316
iteration: 1222 objective: 0.187028
iteration: 1223 objective: 0.186898
iteration: 1224 objective: 0.186595
iteration: 1225 objective: 0.187202
iteration: 1226 objective: 0.186935
iteration: 1227 objective: 0.186631
iteration: 1228 objective: 0

iteration: 1634 objective: 0.178436
iteration: 1635 objective: 0.178212
iteration: 1636 objective: 0.177984
iteration: 1637 objective: 0.178426
iteration: 1638 objective: 0.178801
iteration: 1639 objective: 0.178556
iteration: 1640 objective: 0.178315
iteration: 1641 objective: 0.178081
iteration: 1642 objective: 0.177853
iteration: 1643 objective: 0.177626
iteration: 1644 objective: 0.177396
iteration: 1645 objective: 0.177156
iteration: 1646 objective: 0.176926
iteration: 1647 objective: 0.177280
iteration: 1648 objective: 0.177623
iteration: 1649 objective: 0.178172
iteration: 1650 objective: 0.177942
iteration: 1651 objective: 0.177711
iteration: 1652 objective: 0.178067
iteration: 1653 objective: 0.178477
iteration: 1654 objective: 0.178798
iteration: 1655 objective: 0.178577
iteration: 1656 objective: 0.178345
iteration: 1657 objective: 0.178113
iteration: 1658 objective: 0.178551
iteration: 1659 objective: 0.178325
iteration: 1660 objective: 0.178093
iteration: 1661 objective: 0

...

iteration: 74615 objective: 0.183320
iteration: 74616 objective: 0.183315
iteration: 74617 objective: 0.183310
iteration: 74618 objective: 0.183319
iteration: 74619 objective: 0.183314
iteration: 74620 objective: 0.183309
iteration: 74621 objective: 0.183305
iteration: 74622 objective: 0.183312
iteration: 74623 objective: 0.183322
iteration: 74624 objective: 0.183317
iteration: 74625 objective: 0.183312
iteration: 74626 objective: 0.183319
iteration: 74627 objective: 0.183329
iteration: 74628 objective: 0.183336
iteration: 74629 objective: 0.183345
iteration: 74630 objective: 0.183352
iteration: 74631 objective: 0.183348
iteration: 74632 objective: 0.183342
iteration: 74633 objective: 0.183337
iteration: 74634 objective: 0.183332
iteration: 74635 objective: 0.183327
iteration: 74636 objective: 0.183322
iteration: 74637 objective: 0.183317
iteration: 74638 objective: 0.183312
iteration: 74639 objective: 0.183307
iteration: 74640 objective: 0.183302
iteration: 74641 objective: 0.183311
i

iteration: 75114 objective: 0.183279
iteration: 75115 objective: 0.183274
iteration: 75116 objective: 0.183285
iteration: 75117 objective: 0.183294
iteration: 75118 objective: 0.183289
iteration: 75119 objective: 0.183285
iteration: 75120 objective: 0.183281
iteration: 75121 objective: 0.183276
iteration: 75122 objective: 0.183270
iteration: 75123 objective: 0.183266
iteration: 75124 objective: 0.183260
iteration: 75125 objective: 0.183255
iteration: 75126 objective: 0.183250
iteration: 75127 objective: 0.183245
iteration: 75128 objective: 0.183252
iteration: 75129 objective: 0.183247
iteration: 75130 objective: 0.183242
iteration: 75131 objective: 0.183237
iteration: 75132 objective: 0.183232
iteration: 75133 objective: 0.183242
iteration: 75134 objective: 0.183237
iteration: 75135 objective: 0.183232
iteration: 75136 objective: 0.183240
iteration: 75137 objective: 0.183235
iteration: 75138 objective: 0.183230
iteration: 75139 objective: 0.183225
iteration: 75140 objective: 0.183232
i

iteration: 75614 objective: 0.183434
iteration: 75615 objective: 0.183446
iteration: 75616 objective: 0.183441
iteration: 75617 objective: 0.183435
iteration: 75618 objective: 0.183431
iteration: 75619 objective: 0.183426
iteration: 75620 objective: 0.183421
iteration: 75621 objective: 0.183429
iteration: 75622 objective: 0.183438
iteration: 75623 objective: 0.183451
iteration: 75624 objective: 0.183448
iteration: 75625 objective: 0.183457
iteration: 75626 objective: 0.183465
iteration: 75627 objective: 0.183461
iteration: 75628 objective: 0.183457
iteration: 75629 objective: 0.183467
iteration: 75630 objective: 0.183462
iteration: 75631 objective: 0.183457
iteration: 75632 objective: 0.183464
iteration: 75633 objective: 0.183459
iteration: 75634 objective: 0.183454
iteration: 75635 objective: 0.183461
iteration: 75636 objective: 0.183479
iteration: 75637 objective: 0.183476
iteration: 75638 objective: 0.183471
iteration: 75639 objective: 0.183466
iteration: 75640 objective: 0.183461
i

iteration: 76113 objective: 0.183362
iteration: 76114 objective: 0.183357
iteration: 76115 objective: 0.183353
iteration: 76116 objective: 0.183348
iteration: 76117 objective: 0.183343
iteration: 76118 objective: 0.183351
iteration: 76119 objective: 0.183346
iteration: 76120 objective: 0.183343
iteration: 76121 objective: 0.183338
iteration: 76122 objective: 0.183350
iteration: 76123 objective: 0.183345
iteration: 76124 objective: 0.183340
iteration: 76125 objective: 0.183335
iteration: 76126 objective: 0.183330
iteration: 76127 objective: 0.183339
iteration: 76128 objective: 0.183334
iteration: 76129 objective: 0.183328
iteration: 76130 objective: 0.183323
iteration: 76131 objective: 0.183319
iteration: 76132 objective: 0.183314
iteration: 76133 objective: 0.183323
iteration: 76134 objective: 0.183331
iteration: 76135 objective: 0.183326
iteration: 76136 objective: 0.183320
iteration: 76137 objective: 0.183315
iteration: 76138 objective: 0.183311
iteration: 76139 objective: 0.183306
i

iteration: 76613 objective: 0.183268
iteration: 76614 objective: 0.183275
iteration: 76615 objective: 0.183270
iteration: 76616 objective: 0.183267
iteration: 76617 objective: 0.183262
iteration: 76618 objective: 0.183279
iteration: 76619 objective: 0.183274
iteration: 76620 objective: 0.183281
iteration: 76621 objective: 0.183276
iteration: 76622 objective: 0.183285
iteration: 76623 objective: 0.183281
iteration: 76624 objective: 0.183292
iteration: 76625 objective: 0.183287
iteration: 76626 objective: 0.183294
iteration: 76627 objective: 0.183291
iteration: 76628 objective: 0.183286
iteration: 76629 objective: 0.183293
iteration: 76630 objective: 0.183289
iteration: 76631 objective: 0.183284
iteration: 76632 objective: 0.183291
iteration: 76633 objective: 0.183286
iteration: 76634 objective: 0.183281
iteration: 76635 objective: 0.183277
iteration: 76636 objective: 0.183272
iteration: 76637 objective: 0.183280
iteration: 76638 objective: 0.183289
iteration: 76639 objective: 0.183284
i